In [8]:
import cv2
import re

In [5]:
def extract_every_k_second_frame(path_to_video, path_to_folder, k: int, start_frame_index=0, 
                                 num_frames_per_sec=24):
    '''Функция отвечает за вычленение первого кадра каждой n-ной секунды (кадров т.о. становится меньше, 
    k - это номер секунды).
    Проблема: есть все-таки монтажные черные склейки, и n-ные секунды могут выпадать на них, из-за чего кадры
    могут получаться не очень качественными. Возможно, этого не будет заметно, или же количество черных кадров
    будет минимальным'''
    
    vidcap = cv2.VideoCapture(path_to_video)
    success,image = vidcap.read()
    count = start_frame_index * num_frames_per_sec * k
    while success:
        success,image = vidcap.read()
    #     print('Read a new frame: ', success, ' ', count) #это вероятно можно убрать
        if count % (num_frames_per_sec * k) == 0:
            cv2.imwrite(path_to_folder + "/{}_frame.jpg".format(count // (num_frames_per_sec * k)), image)  
        count += 1
    return print('done')

# Division of the movies into frames and frames renaming

In [6]:
film_path = 'Anna_Karenina_2012.mov'
extract_every_k_second_frame(film_path,'2012', 1)

done


In [9]:
# '''Чанк, чтобы переименовать файлы'''

import os
path = '2012'
files = [f for f in os.listdir(path) if f.endswith('.jpg')]
files.sort(key=lambda file: int(re.sub('\D', '', file)))

for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(['2012_1_', str(index), '.jpg'])))

# Series Preprocessing

## Cut the series' episodes

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [10]:
# Function for episodes with the same timing and opening
def cut_film(directory, start_time_in_sec, end_time_in_sec):
    for file in os.listdir(directory):
        if file.endswith('.mp4') or file.endswith('.mkv') or file.endswith('.avi'):
            ffmpeg_extract_subclip(directory + '/' + file, start_time_in_sec, end_time_in_sec, 
                                   targetname = directory + '/cut_series/'+ file)

In [ ]:
directory_soloviev = '/Users/tonleon/Desktop/Films/Soloviev'
cut_film(directory_soloviev, 70, 3023)

In [ ]:
# Function for the episodes with different timing and opening
def cut_films_in_directory(directory, film_2_cut_start_end):
    for file in os.listdir(directory):
        if file.endswith('.mp4') or file.endswith('.mkv') or file.endswith('.avi'):
            start_time_in_sec, end_time_in_sec = film_2_cut_start_end[file]
            ffmpeg_extract_subclip(directory + '/' + file, start_time_in_sec, end_time_in_sec, 
                                   targetname = directory + '/cut_series/'+ file)

In [ ]:
directory_shahnazarov = '/Users/tonleon/Desktop/Films/Shahnazarov'
series_start_end_in_min_sec = [
    [(1, 11), (42, 43)],
    [(1, 42), (42, 7)],
    [(1, 53), (41, 39)],
    [(1, 50), (41, 2)],
    [(1, 36), (44, 24)],
    [(1, 39), (46, 18)],
    [(1, 24), (44, 10)],
    [(1, 25), (44, 0)]
]

shahnazarov_series_2_time_start_end = {}
for i, (start, end) in enumerate(series_start_end_in_min_sec):
    start_time = start[0]  * 60 + start[1]
    end_time = end[0]  * 60 + end[1]
    shahnazarov_series_2_time_start_end['Anna_0{}.mkv'.format(i + 1)] = (start_time, end_time)

In [ ]:
cut_films_in_directory(directory_shahnazarov, shahnazarov_series_2_time_start_end)

## Extract frames from series

In [ ]:
def extract_every_k_second_frame_from_series(path_to_video, path_to_folder, film_name, k: int, start_frame_index=0, series_num='', num_frames_per_sec=24):
    '''Функция отвечает за вычленение первого кадра каждой n-ной секунды (кадров т.о. становится меньше, 
    k - это номер секунды).
    Проблема: есть все-таки монтажные черные склейки, и n-ные секунды могут выпадать на них, из-за чего кадры
    могут получаться не очень качественными. Возможно, этого не будет заметно, или же количество черных кадров
    будет минимальным'''
    
    vidcap = cv2.VideoCapture(path_to_video)
    success,image = vidcap.read()
    count = start_frame_index * num_frames_per_sec * k
    
    if series_num != '':
        series_num = '_' + str(series_num)
        
    while success:
        success,image = vidcap.read()
        if count % (num_frames_per_sec * k) == 0:
#             cv2.imwrite(path_to_folder + str(film_name) + '_' + str(series_num) +"_{}.jpg".format(count // (num_frames_per_sec * k)), image)  
            cv2.imwrite('{}/{}{}_{}.jpg'.format(path_to_folder,
                                                 film_name,
                                                 series_num,
                                                 count // (num_frames_per_sec * k)), image)  
        count += 1
    return print('done')

In [ ]:
extract_every_k_second_frame('/Users/tonleon/Desktop/Films/Karenina_Italian/Anna Karenina E02.avi', 
                             '/Users/tonleon/Desktop/Films/Karenina_Italian/2013_frames', 1, start_frame_index=5880, num_frames_per_sec=24)

In [ ]:
extract_every_k_second_frame_from_series('/Users/tonleon/Desktop/Films/Shahnazarov/cut_series/Anna_08.mkv',
                             '/Users/tonleon/Desktop/Films/Shahnazarov/2017_frames', 
                             2017, 1, 0, 8)